# Clustering With Howso Engine

## Overview

This example notebook will demonstrate Howso Engine’s ability to calculate pairwise distances for the training cases. This capability is based on Howso Engine’s underlying instance-based learning platform. Distances can be used for a variety of use cases, in particular, for clustering algorithms, such as HDBSCAN. Howso Engine’s feature reduction capabilities will also be used to demonstrate how to eliminate features for clustering. 

In [1]:
import os

import numpy as np
import pandas as pd
import plotly.io as pio
from plotly.subplots import make_subplots
from sklearn.cluster import HDBSCAN
from sklearn.datasets import make_blobs
from sklearn.metrics import adjusted_rand_score

from howso import engine
from howso.utilities import infer_feature_attributes
from howso.visuals import plot_umap

pio.renderers.default = os.getenv("HOWSO_RECIPE_RENDERER", "notebook")

ImportError: cannot import name 'plot_umap' from 'howso.visuals' (c:\Users\JacobBeel\miniconda3\envs\engine-recipes-3.12\Lib\site-packages\howso\visuals\__init__.py)

### Step 1: Generate a simple dataset with noise

A 2-D blob is generated along with an extra noise column. The noise column is used to demonstrate feature reduction.


In [ ]:
blobs = make_blobs(n_samples=100, n_features=2, centers=[(1,0), (1,5), (10,2)], random_state=0)
df = pd.DataFrame(blobs[0], columns=['x', 'y'])
df['target'] = blobs[1]

# Noise
noise = np.random.uniform(df['y'].min(), df['y'].max(), len(df))
df['noise'] = noise

### Step 2: Create the Trainee, Train, and Analyze

For questions about the specific steps of this section, please see the [basic workflow guide](https://docs.howso.com/user_guide/basics/basic_workflow.html).

In [ ]:
# Infer feature attributes
features = infer_feature_attributes(df)

# Specify Context and Action Features
action_features = ['target']

context_features = features.get_names(without=action_features)

# Create the Trainee
t = engine.Trainee(
    features=features,
    overwrite_existing=True
)

# Train
t.train(df)

# Targeted Analysis
t.analyze(context_features=context_features, action_features=action_features)

### Step 3: Inspect Feature Importance

In this example, since we have a target variable, we can inspect the feature contributions and feature [Mean Decrease in Accuracy](https://docs.howso.com/getting_started/terminology.html#mda) (MDA) to understand how important each feature is.

In [ ]:
robust_feature_influences = t.react_aggregate(
    context_features=context_features,
    action_feature=action_features[0],
    details={
        "feature_contributions_robust": True,
        "feature_mda_robust": True
    }
)
robust_feature_influences

We can see that the noise column not only contributes less to the prediction than the other variables, but it also marginally increases (and perhaps lowers) the accuracy. This is expected from the noise column and demonstrates how unimportant features can be highlighted with Howso Engine.

### Step 4: Remove Unuseful Features

Unuseful features like the noise feature can reduce the quality of the clustering so we will remove them in this example. Howso Engine allows use to easily remove the features from the Trainee.

In [ ]:
# Remove the noise feature
t.remove_feature('noise')

# update context_features
context_features.remove('noise')

# Reanalyze with the remaining features
t.analyze(context_features=context_features, action_features=action_features)

## Step 5: Get pairwise distances

By directly using Howso Engine's pairwise distances, we can take advantage of all the benefits of Howso Engine, including the ease of preprocessing and categorical variable handling. 

`get_distances` returns a square matrix of distances that make up the pairwise distances from each case to each other case in the model. The distances are computing using the Howso Engine's internal distance metric which has its parameters tuned within the Analyze call. For more information about the Howso Engine's distance metric, see [<em>Surprisal Driven k-NN for Robust and Interpretable Nonparametric Learning </em> (Banerjee et. al. 2023)](https://arxiv.org/abs/2311.10246).

In [ ]:
# Pairwise distances can be obtained with a call to `get_distances`
x = t.get_distances()

x['distances']

## Step 6a: Cluster

Having access to the full matrix of pairwise distances, users have the ability to cluster with whatever algorithm they prefer (provided it supports a matrix of precomputed pairwise distances as an input).

In the case of this recipe, we demonstrate using the HDBSCAN provided through Scikit-Learn.

In [ ]:
hdb = HDBSCAN(min_cluster_size=15, max_cluster_size=50, min_samples=10, metric='precomputed').fit(x['distances'])
hdbscan_labels = hdb.labels_
df["hdbscan_labels"] = hdbscan_labels

fig = make_subplots(rows=1, cols=2, subplot_titles=["True Clusters", "HDBSCAN Clusters"])
for label, group in df.groupby("target"):
    fig.add_scatter(x=group.x, y=group.y, col=1, row=1, mode="markers", name=label)
for label, group in df.groupby("hdbscan_labels"):
    fig.add_scatter(
        x=group.x, y=group.y, col=2, row=1, mode="markers", name=label,
        marker_color=fig.layout["template"]["layout"]["colorway"][label],
        showlegend=False
    )
fig.update_layout(width=1000)
fig.show()

## Step 6b: Visualize with UMAP

UMAP can be used in tandem with Howso Engine to visualize high-dimensional datasets, similar to how HDBSCAN is used.

In [ ]:
fig = plot_umap(t)
fig.show()

## Step 7: Inspect Accuracy 

We calculate the performance of the clustering by measuring the similarity of the clusters compared to the original labels.

In [ ]:
dist_contribution_acc = adjusted_rand_score(df['target'], hdbscan_labels)
print(f'clustering accuracy score: {dist_contribution_acc}')

# Conclusion

As we demonstrate here, it is quite simple to use the Howso Engine for clustering using the internal distance metric by retrieving the pairwise distance matrix with `get_distances`. This pairwise distance metric can be passed to many popular clustering algorithms.